In [72]:
import os
import torch
from torch import nn
from torch import optim
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle

In [68]:
class LidarBrain(nn.Module):
    
    def __init__(self, inputs_size, hidden1_size, hidden2_size, outputs_size):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(inputs_size, hidden1_size),
            nn.Linear(hidden1_size, hidden2_size),
            nn.Linear(hidden2_size, outputs_size)
        )

    def forward(self, x):

        return self.model(x)

In [69]:
load_path = "C:/Users/nilsm/workspace/MakeAIWork2/project/simpylc/data/data_2rounds"

data_file = pd.read_table(load_path, sep = ",", index_col = 0)


# Create X data and Y data by converting them to torch.tensor
X = torch.Tensor(np.array(data_file.iloc[:, :-2], copy=False))

# We decided to leave out the velocity for now.
Y = torch.Tensor(np.array(data_file.iloc[:, 16:17], copy=False))

print(X.type())
print(Y[0].type())

torch.FloatTensor
torch.FloatTensor


In [71]:
epochs = 10

learningRate = 0.0001

#----------------------------------------

lb = LidarBrain(len(X[0]), 10, 10, len(Y[0]))

print(lb)

#----------------------------------------

losses = []

# Take binary cross entropy as loss function (one output interpreted as binary)
lossFunction = nn.MSELoss()

# Use stochastic gradient descent as optimizer, use weights and biases of model
gradientDescent = torch.optim.SGD(lb.parameters(), lr=learningRate)

#----------------------------------------

for i in range(epochs):
    
    for x, y in zip(X, Y):
    
        # Reset the gradient delta's (dw, db)
        gradientDescent.zero_grad()

        # Forward step
        yhat = lb(x)

        # Compute loss
        loss = lossFunction(yhat, y)

        # Keep track of loss
        losses.append(loss.item())

        # Apply gradient descent (via backpropagation)
        loss.backward()

        # Use w += -step * dw * learnRate
        gradientDescent.step()

#----------------------------------------

# fig, ax = plt.subplots()
# ax.set(xlabel='Epoch', ylabel='Cost', title="Training Cost")

# plt.plot([x for x in range(4 * epochs)], losses, 'red')
# plt.show()

#----------------------------------------

LidarBrain(
  (model): Sequential(
    (0): Linear(in_features=16, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=1, bias=True)
  )
)
tensor(1374.7766, grad_fn=<MseLossBackward0>)
tensor(1105.6484, grad_fn=<MseLossBackward0>)
tensor(988.8847, grad_fn=<MseLossBackward0>)
tensor(820.0394, grad_fn=<MseLossBackward0>)
tensor(645.9219, grad_fn=<MseLossBackward0>)
tensor(411.5591, grad_fn=<MseLossBackward0>)
tensor(239.4619, grad_fn=<MseLossBackward0>)
tensor(416.0886, grad_fn=<MseLossBackward0>)
tensor(124.5662, grad_fn=<MseLossBackward0>)
tensor(53.9752, grad_fn=<MseLossBackward0>)
tensor(0.1869, grad_fn=<MseLossBackward0>)
tensor(28.5469, grad_fn=<MseLossBackward0>)
tensor(4.4233, grad_fn=<MseLossBackward0>)
tensor(47.7477, grad_fn=<MseLossBackward0>)
tensor(7.0162, grad_fn=<MseLossBackward0>)
tensor(1.1826, grad_fn=<MseLossBackward0>)
tensor(9.6396, grad_fn=<MseLossBackward0>)
tensor(0.0155, grad_fn=<Ms

In [77]:
save_path = "C:/Users/nilsm/workspace/MakeAIWork2/project/simpylc/pickle/LidarBrain.pkl"

pickle.dump(lb, open(save_path, 'wb'))